In [217]:
import numpy as np
import symforce

symforce.set_backend("sympy")
symforce.set_log_level("warning")
from symforce.notebook_util import display, print_expression_tree

from symforce import sympy as sm
from symforce import geo
from symforce import typing as T
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer
from symforce.values import Values

from symforce.opt.barrier_functions import (
    min_max_power_barrier,
    min_max_linear_barrier,
    max_linear_barrier
)

import numpy as np
import matplotlib.pyplot as plt

In [218]:
t = sm.Symbol("t")
v_0 = sm.Symbol("v_0")
v_b = sm.Symbol("v_b")
T_0 = sm.Symbol("T_0")
T_a = sm.Symbol("T_a")  # arrival time
L = sm.Symbol("L") # drive length
a = sm.Symbol("a")
sl = sm.Symbol("sl")

In [219]:
v = -1/2 * (v_0 - v_b) * sm.tanh(a * (t - T_0)) + (v_0 + v_b) * 0.5 - sl * t
v

In [220]:
alpha = sm.diff(v, t)
alpha

In [221]:
L = sm.integrate(v, (t, 0, T_a))
L

    2                               ⎛⎧⎛      log(tanh(T₀⋅a - Tₐ⋅a) + 1)⎞      
  Tₐ ⋅sl                            ⎜⎪⎜-Tₐ - ──────────────────────────⎟⋅(-0.5
- ────── + 0.5⋅Tₐ⋅v₀ + 0.5⋅Tₐ⋅v_b + ⎜⎨⎝                  a             ⎠      
    2                               ⎜⎪                                        
                                    ⎝⎩                                        

                 (-0.5⋅v₀ + 0.5⋅v_b)⋅log(tanh(T₀⋅a) + 1)           ⎞
⋅v₀ + 0.5⋅v_b) + ───────────────────────────────────────  for a ≠ 0⎟
                                    a                              ⎟
                                                                   ⎟
       0                                                  otherwise⎠

In [222]:
def velocity_function(t: T.Scalar, v_0: T.Scalar, v_b: T.Scalar, a: T.Scalar, T_0: T.Scalar, sl: T.Scalar) -> T.Scalar :
    return -1/2 * (v_0 - v_b) * sm.tanh(a * (t - T_0)) + (v_0 + v_b) * 0.5 - sl * t

def acceleration_function(t: T.Scalar, v_0: T.Scalar, v_b: T.Scalar, a: T.Scalar, T_0: T.Scalar, sl: T.Scalar) -> T.Scalar :
    return sm.diff(velocity_function(t, v_0, v_b, a, T_0, sl), t)

def length_function(v_0: T.Scalar, v_b: T.Scalar, a: T.Scalar, T_0: T.Scalar, T_a: T.Scalar, sl: T.Scalar, eps: T.Scalar) -> T.Scalar:
    # t_z = sm.Symbol("t_z")
    # length = sm.integrate(velocity_function(t_z, v_0, v_b, a, T_0), (t_z, 0, T_a))
    length = (-T_a - sm.log(sm.tanh(T_0 * a - T_a * a)+1)/(a+eps)) * (-0.5 * v_0 + 0.5 * v_b) + (-0.5 * v_0 + 0.5 * v_b) * sm.log(sm.tanh(T_0 *a) + 1) / (a+eps) + 0.5 * T_a * v_0 + 0.5 * T_a * v_b - T_a**2 * sl * 0.5
    return length

In [265]:
def initial_velocity_residual(
        v_0: T.Scalar, v_b: T.Scalar, a: T.Scalar, T_0: T.Scalar, sl: T.Scalar
) -> geo.V1:
    return geo.V1(v_0 - velocity_function(0, v_0, v_b, a, T_0, sl))

def end_velocity_residual(
        T_a: T.Scalar, v_0: T.Scalar, v_b: T.Scalar, a: T.Scalar, T_0: T.Scalar, sl: T.Scalar
) -> geo.V1:
    return geo.V1(velocity_function(T_a, v_0, v_b, a, T_0, sl))



def drive_length_residual(
    drive_length: T.Scalar, T_a: T.Scalar, v_0: T.Scalar, v_b: T.Scalar, a: T.Scalar, T_0: T.Scalar, sl: T.Scalar, eps: T.Scalar
) -> geo.V1:
    return geo.V1(drive_length - length_function(v_0, v_b, a, T_0, T_a, sl, eps))

def acceleration_residual(
        aim_acc: T.Scalar, check_time: T.Scalar, v_0: T.Scalar, v_b: T.Scalar, a: T.Scalar, T_0: T.Scalar, sl: T.Scalar
) -> geo.V1:
    return geo.V1(aim_acc - acceleration_function(check_time, v_0, v_b, a, T_0, sl))

def sl_constraint(sl: T.Scalar) -> geo.V1:
    return geo.V1(max_linear_barrier(-sl, 0.1, 1000, 0.1))

def v0_vb_constraint(v_0: T.Scalar, v_b: T.Scalar) ->geo.V1:
    return geo.V1(max_linear_barrier(v_b, 0.0, 100, 0.1))

In [266]:
factors = []

factors.append(
    Factor(
        residual=initial_velocity_residual,
        keys=["v_0", "v_b", "a", "T_0", "sl"],
    )
)

factors.append(
    Factor(
        residual=end_velocity_residual,
        keys=["T_a", "v_0", "v_b", "a", "T_0", "sl"],
    )
)

factors.append(Factor(
    residual=drive_length_residual,
    keys=["L", "T_a", "v_0", "v_b", "a", "T_0", "sl", "eps"],
))

# initial acceleration
factors.append(Factor(
    residual=acceleration_residual,
    keys=["initial_acc","T_a", "v_0", "v_b", "a", "T_0", "sl"]
))

# end acceleration
factors.append(Factor(
    residual=acceleration_residual,
    keys=["end_acc","T_a", "v_0", "v_b", "a", "T_0", "sl"]
))

factors.append(Factor(
    residual=sl_constraint,
    keys=["sl"]
))


factors.append(Factor(
    residual=v0_vb_constraint,
    keys=["v_0", "v_b"]
))



In [289]:
optimizer = Optimizer(
    factors=factors,
    # optimized_keys=["v_b", "a", "T_0", "sl"],
    optimized_keys=["v_b", "T_0", "sl"],
    # So that we save more information about each iteration, to visualize later:
    debug_stats=True,
)

In [295]:
initial_speed = 1.38
drive_length = 30
arrival_time = 50

initial_values = Values(
    v_0 = initial_speed,
    initial_acc = -1,
    end_acc = -1.9,
    L = drive_length,
    T_a = arrival_time,
    eps = sm.default_epsilon,
    v_b = -0.5,
    a = 1,
    T_0 = arrival_time*0.5,
    sl = 0,
)

result = optimizer.optimize(initial_values)
print(result)

opt_v_b  = result.optimized_values["v_b"]
opt_a = result.optimized_values["a"]
opt_T_0 = result.optimized_values["T_0"]
opt_sl = result.optimized_values["sl"]


opt_t = np.linspace(0, arrival_time, 100)
opt_v = [velocity_function(c_t, initial_speed, opt_v_b, opt_a, opt_T_0, opt_sl) for c_t in opt_t]

# plt.ylim([0, initial_speed])
plt.plot(opt_t, opt_v)


ValueError: math domain error